# ## Install Dependencies


In [66]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [67]:
# Verify installations
import numpy as np
import sklearn
import hazm
import json

# Persian Text Retrieval System
# This notebook implements a simple Persian text retrieval system using the `IR_data_news_12k.json` dataset. The system uses TF-IDF for ranking documents and evaluates performance using precision and recall.

# ## 1. Load the Dataset
# Load the `IR_data_news_12k.json` dataset.


In [68]:
def read_json(path):
  file = open(path)
  data = json.load(file)
  return data

In [69]:
input_data = read_json('./data/IR_data_news_12k.json')

In [70]:
print(list(input_data.values())[0])

{'title': 'اعلام زمان قرعه کشی جام باشگاه های فوتسال آسیا', 'content': '\nبه گزارش خبرگزاری فارس، کنفدراسیون فوتبال آسیا (AFC) در نامه ای رسمی به فدراسیون فوتبال ایران و باشگاه گیتی پسند زمان\xa0 قرعه کشی جام باشگاه های فوتسال آسیا را رسماً اعلام کرد. بر این اساس 25 فروردین ماه 1401 مراسم قرعه کشی جام باشگاه های فوتسال آسیا در مالزی برگزار می شود. باشگاه گیتی پسند بعنوان قهرمان فوتسال ایران در سال 1400 به این مسابقات راه پیدا کرده است. پیش از این گیتی پسند تجربه 3 دوره حضور در جام باشگاه های فوتسال آسیا را داشته که هر سه دوره به فینال مسابقات راه پیدا کرده و یک عنوان قهرمانی و دو مقام دومی بدست آورده است. انتهای پیام/\n\n\n', 'tags': ['اعلام زمان', 'قرعه\u200cکشی', 'قرعه\u200cکشی جام', 'قرعه\u200cکشی جام باشگاه\u200cهای فوتسال', 'ای اف سی', 'گیتی پسند'], 'date': '3/15/2022 5:59:27 PM', 'url': 'https://www.farsnews.ir/news/14001224001005/اعلام-زمان-قرعه-کشی-جام-باشگاه-های-فوتسال-آسیا', 'category': 'sports'}


In [71]:
print(list(input_data.values())[0].keys())
contents = [input_data[i]['content'] for i in input_data]
print(len(contents))
print(contents[0])

dict_keys(['title', 'content', 'tags', 'date', 'url', 'category'])
12202

به گزارش خبرگزاری فارس، کنفدراسیون فوتبال آسیا (AFC) در نامه ای رسمی به فدراسیون فوتبال ایران و باشگاه گیتی پسند زمان  قرعه کشی جام باشگاه های فوتسال آسیا را رسماً اعلام کرد. بر این اساس 25 فروردین ماه 1401 مراسم قرعه کشی جام باشگاه های فوتسال آسیا در مالزی برگزار می شود. باشگاه گیتی پسند بعنوان قهرمان فوتسال ایران در سال 1400 به این مسابقات راه پیدا کرده است. پیش از این گیتی پسند تجربه 3 دوره حضور در جام باشگاه های فوتسال آسیا را داشته که هر سه دوره به فینال مسابقات راه پیدا کرده و یک عنوان قهرمانی و دو مقام دومی بدست آورده است. انتهای پیام/





# ## 2. Preprocess the Data
# Use the `Hazm` library to normalize, tokenize, and remove stop words from the text.


In [72]:
from parsivar import Normalizer, Tokenizer, FindStems
from hazm import stopwords_list

In [73]:
normalizer = Normalizer()
tokenizer = Tokenizer()
stemmer = FindStems()

In [74]:
stopwords = {stopwords_list()[i] for i in range(0, len(stopwords_list()) - 1)}
extra_stopwords = ['،', '.', ')', '(', '}', '{', '«', '»', '؛', ':',  '؟','>','<','|','+','-','*',"^",'%','#','=','_','/','«','»','$','[',']','&',"❊",'«','»']
stopwords.update(extra_stopwords)

In [75]:
def preprocess(contents, rm_sw=True, stemming=True):
  preprocessed_docs = []
  for content in contents:
    
    # normalizing
    normalized_content = normalizer.normalize(content)
    content_tokens = tokenizer.tokenize_words(normalized_content)
    tokens = []
    for token in content_tokens:
      # stemming
      if stemming:
        token = stemmer.convert_to_stem(token)
      # remove stopwords
      if rm_sw:
          if token in stopwords:
                continue
      tokens.append(token)
    preprocessed_docs.append(tokens)
    # tokens of each doc
  return preprocessed_docs

In [76]:
preprocessed_docs = preprocess(contents)

In [77]:
print(len(preprocessed_docs))

12202


# ## 3. Positional Indexing

In [102]:
class Term:
    def __init__(self):
        self.total_freq = 0
        self.pos_in_doc = {}  # {doc_id: [positions]}
        self.freq_in_doc = {} # {doc_id: frequency}
        self.weight_in_doc = {} # {doc_id: tf-idf weight}
        self.champ_list = {}   # Top r documents

    def update_posting(self, doc_id, term_position):
        if doc_id not in self.pos_in_doc:
            self.pos_in_doc[doc_id] = []
            self.freq_in_doc[doc_id] = 0
        self.pos_in_doc[doc_id].append(term_position)
        self.freq_in_doc[doc_id] += 1
        self.total_freq += 1

    # Calculate TF-IDF weight for a document
    def calc_weight(self, doc_id, collection_size):
        tf = 1 + np.log10(self.freq_in_doc[doc_id]) if self.freq_in_doc[doc_id] > 0 else 0
        idf = np.log10(collection_size / len(self.freq_in_doc))
        self.weight_in_doc[doc_id] = tf * idf

    # Create champion list with top r documents by weight
    def create_champ_list(self, r):
        sorted_docs = sorted(self.weight_in_doc.items(), key=lambda x: x[1], reverse=True)
        self.champ_list = dict(sorted_docs[:r])

    def get_champ_list(self):
        return self.champ_list


In [103]:
def positional_indexing(preprocessed_docs):
    p_inv_index = {} 
    for doc_id in range(len(preprocessed_docs)):
        for pos in range(len(preprocessed_docs[doc_id])):
            term = preprocessed_docs[doc_id][pos]
            if term in p_inv_index:
                term_obj = p_inv_index[term]
            else:
                term_obj = Term()
            term_obj.update_posting(doc_id, pos)
            p_inv_index[term] = term_obj

    return p_inv_index

In [104]:
positional_index = positional_indexing(preprocessed_docs)

In [105]:
print(positional_index['بایرن'].pos_in_doc)

{98: [49], 101: [46], 110: [29, 57], 142: [335], 150: [73], 178: [9, 30, 40], 222: [125], 225: [44], 300: [27, 56], 313: [218], 316: [125], 357: [74], 370: [144], 384: [5, 34], 386: [5, 70], 405: [12], 413: [72], 434: [5, 24], 443: [11, 25, 37, 67], 444: [77], 492: [108], 583: [57], 706: [17], 712: [18], 724: [13, 73, 88, 117], 747: [9, 42, 53], 759: [9], 760: [6, 34, 52], 791: [32], 813: [10, 25, 59, 103], 836: [43, 52], 863: [38, 51, 96], 934: [55], 948: [45, 64], 968: [71, 86], 1152: [38], 1156: [29], 1245: [8, 16, 22], 1285: [15, 26, 35, 55, 91, 101], 1303: [33, 205], 1313: [94], 1421: [37], 1636: [24], 1637: [9, 17, 28], 1701: [99], 1716: [64, 75], 1723: [8, 39], 1954: [39], 1996: [29], 2005: [5, 31, 42], 2013: [60, 77, 92], 2064: [10, 35, 46, 62, 68], 2201: [39, 47], 2306: [9, 35, 41], 2363: [10, 31, 55, 78, 84, 99, 133, 143, 178, 184, 189, 198, 205, 211, 218, 223, 226, 248, 268, 275, 311, 324, 336, 351], 2421: [48], 2509: [25], 2516: [74], 2518: [223, 241], 2526: [63], 2629: [22

In [106]:
preprocessed_docs[222]

['گزارش',
 'خبرگزاری',
 'فارس',
 'سایت',
 'weallfollowunited',
 'انگلیس',
 'گزارش',
 'حضور',
 'نماینده',
 'باشگاه',
 'منچستر',
 'یونایتد',
 'بازی',
 'پورتو',
 'لیون',
 'بازی',
 'مرحله',
 'هشتم',
 'نهایی',
 'لیگ',
 'اروپا',
 'خبر',
 'بازی',
 'پورتو',
 'لیون',
 'نتیجه',
 'صفر',
 'نفع',
 'تیم',
 'فرانسوی',
 'پایان',
 'مهدی',
 'طارم',
 'ستاره',
 'تیم',
 'ملی',
 'کشور',
 'بازی',
 'برخلاف',
 'بازی',
 'گذشته',
 'توانست&توان',
 'عمل\u200cکرد',
 'رسانه',
 'انگلیسی',
 'گزارش',
 'عنوان',
 'استعدادیاب\u200cهای',
 'باشگاه',
 'منچستر',
 'ورزشگاه',
 'پورتو',
 'حضور',
 'کرد&کن',
 'ستاره',
 'تیم',
 'زیرنظر',
 'گرفت&گیر',
 'تابستان',
 'اقداماتی',
 'جذب',
 'آن\u200cها',
 'انجام',
 'شد&شو',
 'مهدی',
 'طارم',
 'ستاره',
 'ایرانی',
 'یکی',
 'ستارگانی\u200cبود',
 'رادار',
 'نماینده',
 'منچستر',
 'یونایتد',
 'قرار',
 'گرفت&گیر',
 'توانست&توان',
 'عمل\u200cکرد',
 'درخشانید&درخشان',
 'مقابل',
 'دیده',
 'مسئول',
 'شیاطین',
 'سرخ',
 'رسانه',
 'نوشت&نویس',
 'منچستریونایتد',
 'نماینده',
 'حضور',
 'بازی',
 'پورتو',
 

In [107]:
# check the value of total_freq is correct or not
print(positional_index['مونیخ'].total_freq)
print(sum(positional_index['مونیخ'].freq_in_doc.values()))

208
208


## 4. Answering Query

In [108]:
from itertools import permutations
import re

In [109]:
def process_phrase(tokens):

    result = []
# used when we have more than 2 words in our phrase
# split it to 2 biword index
# aggregate the results
    for biword in permutations(tokens, 2):
        w1 = biword[0]
        w2 = biword[1]
        if (w1 not in positional_index.keys()) or (w2 not in positional_index.keys()):
            return []
        
        indx1 = tokens.index(w1)
        indx2 = tokens.index(w2)
        pos_dic_1 = positional_index.get(w1).pos_in_doc
        pos_dic_2 = positional_index.get(w2).pos_in_doc  
        k = abs(indx1-indx2)

        docs = positional_intersect(pos_dic_1, pos_dic_2, k)
        
        if len(result) == 0:
            result = docs
        else:
            result = list(set(result) & set(docs))

    return result

In [110]:
def process_query(not_words=[], phrases=[], words=[]):
    ranks={}
    
    # find words
    for token in words:
        if token in positional_index.keys():
            for doc_id in positional_index[token].pos_in_doc.keys():
                if doc_id in ranks.keys():
                    ranks[doc_id]+=1
                else:
                    ranks[doc_id]=1
    # find phrases
    for phrase in phrases:
        for doc_id in process_phrase(phrase):
            if doc_id in ranks.keys():
                ranks[doc_id] += 1
            else:
                ranks[doc_id] = 1
    # find ! not words
    not_words_docs = []
    for word in not_words:
        doc_ids = positional_index[word].pos_in_doc.keys()
        for doc_id in doc_ids:
            not_words_docs.append(doc_id)
            
    # from results remove docs which contain not
    if len(ranks) > 0:
        for doc in not_words_docs:
            if doc in ranks.keys():
                del ranks[doc]
                
    ranks = dict(sorted(ranks.items(), key=lambda x: x[1], reverse=True))
    
    return ranks

def not_terms(query):
    splitted_query = query.split()
    indices = [i for i in range(len(splitted_query)) if splitted_query[i]=='!']
    result = [splitted_query[i+1] for i in indices]
    return result

def get_phrase(query):
    res = []
    quoted = re.compile('"[^"]*"')
    for value in quoted.findall(query):
        value = value.replace('"', '').strip().split()
        res.append(value)
    return res

def search_query(query):
    # preprocessed query
    query = ' '.join(preprocess([query], True, True)[0])
    phrases = get_phrase(query)
    flat_phrases = [item for sublist in phrases for item in sublist]
    not_words = not_terms(query)
    query = query.replace('"','')
    query = query.replace('!', '')
    splitted_query = query.split()
    looking_words = []
    for x in splitted_query:
      if x not in not_words and x not in flat_phrases:
        looking_words.append(x)
    output = process_query(not_words=not_words, phrases=phrases, words=looking_words)  
    return output

def print_output(output_dict):
    ids = list(output_dict.keys())[:5]
    for i in range(len(ids)):
        print(f'Rank {i + 1}:')
        title = input_data[str(ids[i])]['title']
        url = input_data[str(ids[i])]['url']
        print('title: ', title, '\nurl: ', url)
        print('------------')

In [111]:
query = 'تحریم‌های آمریکا علیه ایران'
res = search_query(query)
print_output(res)

Rank 1:
title:  خبرگزاری فارس ۱۹ ساله شد 
url:  https://www.farsnews.ir/news/14001122000809/خبرگزاری-فارس-۱۹-ساله-شد
------------
Rank 2:
title:  اصولی: فدراسیون فوتبال جمهوری اسلامی ایران هستیم نه جزیره مستقل/ با گفتار ساختارشکنانه فدراسیون را به ناکجا آباد می‌برند 
url:  https://www.farsnews.ir/news/14001117000518/اصولی-فدراسیون-فوتبال-جمهوری-اسلامی-ایران-هستیم-نه-جزیره-مستقل-با
------------
Rank 3:
title:  احتمال مبادله نازنین زاغری در ازای 530میلیون دلار 
url:  https://www.farsnews.ir/news/14001223001080/احتمال-مبادله-نازنین-زاغری-در-ازای-530میلیون-دلار
------------
Rank 4:
title:  متکی: آمریکا با ابزار ناتو به دنبال تجزیه روسیه است 
url:  https://www.farsnews.ir/news/14001222000749/متکی-آمریکا-با-ابزار-ناتو-به-دنبال-تجزیه-روسیه-است
------------
Rank 5:
title:  توضیحات یک منبع آگاه درباره وقفه مذاکرات وین 
url:  https://www.farsnews.ir/news/14001222000450/توضیحات-یک-منبع-آگاه-درباره-وقفه-مذاکرات-وین
------------


In [112]:
query = 'تحریم‌های آمریکا ! ایران'
res = search_query(query)
print_output(res)

Rank 1:
title:  ادامه تحریم‌های سیاسی علیه المپیک پکن/ژاپن هم به صف منتقدان پیوست 
url:  https://www.farsnews.ir/news/14001003000306/ادامه-تحریم‌های-سیاسی-علیه-المپیک-پکن-ژاپن-هم-به-صف-منتقدان-پیوست
------------
Rank 2:
title:  انتقاد دانشجویان ایرانی در اروپا به برخورد دوگانه مدعیان حقوق بشر با قضایای اوکراین و جنایت‌های آل سعود 
url:  https://www.farsnews.ir/news/14001224000014/انتقاد-دانشجویان-ایرانی-در-اروپا-به-برخورد-دوگانه-مدعیان-حقوق-بشر-با
------------
Rank 3:
title:  محو رژیم صهیونیستی از آرمان‌های نظام اسلامی حذف نشده است 
url:  https://www.farsnews.ir/news/14001222000379/محو-رژیم-صهیونیستی-از-آرمان‌های-نظام-اسلامی-حذف-نشده-است
------------
Rank 4:
title:  تجربه نشان داده به عهد آمریکا در مذاکرات نمی‌شود اعتماد کرد 
url:  https://www.farsnews.ir/news/14001203000366/تجربه-نشان-داده-به-عهد-آمریکا-در-مذاکرات-نمی‌شود-اعتماد-کرد
------------
Rank 5:
title:  سود مافیای اسلحه‌سازی آمریکا در ناامن بودن جهان است 
url:  https://www.farsnews.ir/news/14001211000898/سود-مافیای-اسلحه‌سازی-

In [113]:
query = 'کنگره ضدتروریست'
res = search_query(query)
print_output(res)

Rank 1:
title:  توضیحات یک منبع آگاه درباره وقفه مذاکرات وین 
url:  https://www.farsnews.ir/news/14001222000450/توضیحات-یک-منبع-آگاه-درباره-وقفه-مذاکرات-وین
------------
Rank 2:
title:  بحران دوباره گریبان وزنه‌برداری را گرفت/زیرپا گذاشتن قوانین در IWF 
url:  https://www.farsnews.ir/news/14001223000130/بحران-دوباره-گریبان-وزنه‌برداری-را-گرفت-زیرپا-گذاشتن-قوانین-در-IWF
------------
Rank 3:
title:  برگزاری مراسم روز درختکاری در فدراسیون ووشو/ ملی‌پوشان ۷۲ اصله نهال را غرس کردند 
url:  https://www.farsnews.ir/news/14001215000800/برگزاری-مراسم-روز-درختکاری-در-فدراسیون-ووشو-ملی‌پوشان-۷۲-اصله-نهال-را
------------
Rank 4:
title:  «پهلوانان ماندگار؛ ۵۱۳۵ شهید ورزشکار به نیت هر شهید یک درخت»/ کاشت نمادین درخت در فوتبال 
url:  https://www.farsnews.ir/news/14001215000248/پهلوانان-ماندگار-۵۱۳۵-شهید-ورزشکار-به-نیت-هر-شهید-یک-درخت-کاشت-نمادین
------------
Rank 5:
title:  برگزاری سوپرجام فوتبال کشور به نام شهدای چوار 
url:  https://www.farsnews.ir/news/14001019000298/برگزاری-سوپرجام-فوتبال-کشور-به-نا

In [114]:
import math
import numpy as np

In [115]:
def calculate_tf_for_query(term, tokens):
    freq = 0
    for token in tokens:
        if token == term:
            freq += 1
    if freq > 0:
        return 1 + math.log10(freq)
    return 0

In [124]:
def calculate_tf(term, doc_id):
    freq = term.freq_in_doc[doc_id]
    if freq > 0:  
        return 1 + math.log10(freq)
    return 0

def calculate_idf(term, collection_size):
    n = len(term.freq_in_doc)
    return math.log10(collection_size/n) if n > 0 else 0

def calculate_tf_idf(term, doc_id, collection_size):
  return calculate_tf(term, doc_id) * calculate_idf(term, collection_size)

In [125]:
def calculate_weights(dictionary, collection_size):
    for term in dictionary:
        postings_list = dictionary[term].freq_in_doc.keys()  # Get document IDs
        for doc_id in postings_list:
            dictionary[term].calc_weight(doc_id, collection_size)


In [126]:
calculate_weights(positional_index, len(contents))

positional_index['فوتبال'].pos_in_doc

{0: [4, 10],
 1: [27, 34],
 2: [34],
 3: [17, 38, 64, 75, 97, 119, 145],
 7: [8],
 9: [35],
 12: [6, 48],
 15: [6],
 16: [6, 14],
 23: [32, 68],
 29: [13, 209],
 33: [4, 9, 25, 43],
 35: [4, 32],
 36: [15, 27],
 37: [6, 15],
 40: [101],
 42: [34],
 45: [22],
 48: [303],
 53: [56, 201, 236, 247, 283, 284, 420],
 63: [34],
 64: [75],
 66: [12, 37, 89, 181, 210],
 69: [8, 34],
 74: [9],
 76: [57],
 79: [18],
 80: [9, 35, 85],
 81: [11, 45, 46, 73, 87, 97, 100, 105, 116, 120, 135, 142, 165],
 82: [15],
 84: [37],
 85: [11, 26, 70, 89, 139, 184, 249, 356, 368],
 86: [210, 554],
 87: [5, 49],
 88: [12],
 89: [10, 110, 207, 264, 282, 328, 397, 401, 410, 420],
 90: [4],
 91: [272],
 94: [10],
 95: [47],
 96: [16, 31, 48],
 97: [5],
 102: [10],
 103: [4],
 104: [104],
 107: [8],
 109: [10],
 112: [40],
 113: [8],
 115: [8],
 120: [23, 31, 51, 73],
 123: [10],
 127: [4, 13, 26],
 128: [124],
 129: [30, 31, 54, 58, 72, 91, 112, 117, 122],
 134: [33, 99, 105, 117, 126, 131, 148, 162],
 136: [7, 27

In [127]:
calculate_weights(positional_index, len(contents))

positional_index['فوتبال'].pos_in_doc

{0: [4, 10],
 1: [27, 34],
 2: [34],
 3: [17, 38, 64, 75, 97, 119, 145],
 7: [8],
 9: [35],
 12: [6, 48],
 15: [6],
 16: [6, 14],
 23: [32, 68],
 29: [13, 209],
 33: [4, 9, 25, 43],
 35: [4, 32],
 36: [15, 27],
 37: [6, 15],
 40: [101],
 42: [34],
 45: [22],
 48: [303],
 53: [56, 201, 236, 247, 283, 284, 420],
 63: [34],
 64: [75],
 66: [12, 37, 89, 181, 210],
 69: [8, 34],
 74: [9],
 76: [57],
 79: [18],
 80: [9, 35, 85],
 81: [11, 45, 46, 73, 87, 97, 100, 105, 116, 120, 135, 142, 165],
 82: [15],
 84: [37],
 85: [11, 26, 70, 89, 139, 184, 249, 356, 368],
 86: [210, 554],
 87: [5, 49],
 88: [12],
 89: [10, 110, 207, 264, 282, 328, 397, 401, 410, 420],
 90: [4],
 91: [272],
 94: [10],
 95: [47],
 96: [16, 31, 48],
 97: [5],
 102: [10],
 103: [4],
 104: [104],
 107: [8],
 109: [10],
 112: [40],
 113: [8],
 115: [8],
 120: [23, 31, 51, 73],
 123: [10],
 127: [4, 13, 26],
 128: [124],
 129: [30, 31, 54, 58, 72, 91, 112, 117, 122],
 134: [33, 99, 105, 117, 126, 131, 148, 162],
 136: [7, 27

In [128]:
positional_index['فوتبال'].weight_in_doc

{0: 0.7366556536239535,
 1: 0.7366556536239535,
 2: 0.5662095847744086,
 3: 1.0447121951045477,
 7: 0.5662095847744086,
 9: 0.5662095847744086,
 12: 0.7366556536239535,
 15: 0.5662095847744086,
 16: 0.7366556536239535,
 23: 0.7366556536239535,
 29: 0.7366556536239535,
 33: 0.9071017224734984,
 35: 0.7366556536239535,
 36: 0.7366556536239535,
 37: 0.7366556536239535,
 40: 0.5662095847744086,
 42: 0.5662095847744086,
 45: 0.5662095847744086,
 48: 0.5662095847744086,
 53: 1.0447121951045477,
 63: 0.5662095847744086,
 64: 0.5662095847744086,
 66: 0.9619731006992724,
 69: 0.7366556536239535,
 74: 0.5662095847744086,
 76: 0.5662095847744086,
 79: 0.5662095847744086,
 80: 0.8363602122962736,
 81: 1.1969349877462756,
 82: 0.5662095847744086,
 84: 0.5662095847744086,
 85: 1.1065108398181385,
 86: 0.7366556536239535,
 87: 0.7366556536239535,
 88: 0.5662095847744086,
 89: 1.1324191695488173,
 90: 0.5662095847744086,
 91: 0.5662095847744086,
 94: 0.5662095847744086,
 95: 0.5662095847744086,
 96: 0

In [129]:
def champions_list(dictionary, r):
    for term in dictionary:
        dictionary[term].create_champ_list(r)
        
champions_list(positional_index, 50)

positional_index['فوتبال'].get_champ_list()

{3457: 1.5425111089376038,
 4918: 1.473311307247907,
 2098: 1.4404757110294113,
 163: 1.418439929022133,
 6323: 1.4025697970706823,
 1950: 1.337232937381684,
 5268: 1.3263021734744984,
 5395: 1.3263021734744984,
 6128: 1.3148628226264127,
 1322: 1.302865238398362,
 2286: 1.2902521301986898,
 2287: 1.2902521301986898,
 2788: 1.2902521301986898,
 6350: 1.2902521301986898,
 2626: 1.2769569086676833,
 536: 1.2629015576341203,
 2109: 1.2629015576341203,
 5252: 1.2629015576341203,
 5303: 1.2629015576341203,
 5353: 1.2629015576341203,
 6327: 1.2629015576341203,
 6673: 1.2629015576341203,
 1643: 1.2479938601725882,
 4926: 1.2479938601725882,
 5102: 1.2479938601725882,
 1276: 1.2321237282211375,
 1572: 1.2321237282211375,
 2391: 1.2321237282211375,
 3057: 1.2321237282211375,
 3109: 1.2321237282211375,
 4962: 1.2321237282211375,
 5583: 1.2321237282211375,
 5991: 1.2321237282211375,
 6069: 1.2321237282211375,
 142: 1.2151582639540925,
 246: 1.2151582639540925,
 860: 1.2151582639540925,
 1974: 1.2

In [130]:
def get_docs_norm(dictionary, collection_size):
    lengths = np.zeros(collection_size)
    for term in dictionary:
        for doc_id, weight in dictionary[term].weight_in_doc.items():
            lengths[int(doc_id)] += weight ** 2
    return np.sqrt(lengths)

docs_norms = get_docs_norm(positional_index, len(contents))
print(docs_norms)

[10.33726671  7.70131393  7.14326871 ... 16.20218927 18.79565458
  8.16871913]


In [136]:
def cosine_similarity(query_tokens, dictionary, collection_size, norms):
  scores = {}
  for term in query_tokens:
    w_t_q = calculate_tf_for_query(term, query_tokens) * calculate_idf(dictionary[term], collection_size)
    for doc_id, w_t_d in dictionary[term].get_champ_list().items():
      if doc_id not in scores:
        scores[doc_id] = 0
      scores[doc_id] += w_t_q * w_t_d 
      
  for doc_id in scores:
    scores[doc_id] /= norms[int(doc_id)]
  return scores

def print_result(scores, k):
    sorted_scores = dict(sorted(scores.items(), key=lambda item: item[1],reverse=True)[:k])
    for doc_id in sorted_scores:
        title = input_data[str(doc_id)]['title']
        url = input_data[str(doc_id)]['url']
        print(doc_id, 'title: ', title, '\nurl: ', url)
        print('------------')

In [137]:
query = 'فوتبال'
query_tokens = preprocess([query], True, True)[0]
scores = cosine_similarity(query_tokens, positional_index, len(contents), docs_norms)
print_result(scores, k=5)

1466 title:  نکونام: نفتی ها بهترین بازی خود را انجام دادند/بازیکنان جدیدمان کیفیت بالای خود را نشان دادند 
url:  https://www.farsnews.ir/news/14001204001165/نکونام-نفتی-ها-بهترین-بازی-خود-را-انجام-دادند-بازیکنان-جدیدمان-کیفیت
------------
81 title:  ماجدی: فوتبال کشور به تغییرات نیاز دارد 
url:  https://www.farsnews.ir/news/14001223000539/ماجدی-فوتبال-کشور-به-تغییرات-نیاز-دارد
------------
6690 title:  امیدواری ملی پوش سابق فوتبال ساحلی بابت تغییرات در کادرفنی تیم ملی 
url:  https://www.farsnews.ir/news/14000927000275/امیدواری-ملی-پوش-سابق-فوتبال-ساحلی-بابت-تغییرات-در-کادرفنی-تیم-ملی
------------
139 title:  تقدیر مربی تیم فوتبال خلیج فارس از ماجدی بابت رسیدگی به شائبه تبانی در لیگ جوانان 
url:  https://www.farsnews.ir/news/14001222000297/تقدیر-مربی-تیم-فوتبال-خلیج-فارس-از-ماجدی-بابت-رسیدگی-به-شائبه-تبانی-در
------------
860 title:  دیدار مدیر تیم ملی ایران با سفیر کره جنوبی/ هیان: امیدوارم مقابل ایران مساوی کنیم 
url:  https://www.farsnews.ir/news/14001213000138/دیدار-مدیر-تیم-ملی-ای

In [133]:
query = 'تیم ملی فوتبال'
query_tokens = preprocess([query], True, True)[0]
scores = cosine_similarity(query_tokens, positional_index, len(contents), docs_norms)
print_result(scores, k=5)

142 title:  اسکوچیچ: مردم متوجه شده‌اند که می‌توانند هدایت تیم ملی را به من واگذار کنند/هاشمیان سواد اروپایی از فوتبال دارد 
url:  https://www.farsnews.ir/news/14001222000329/اسکوچیچ-مردم-متوجه-شده‌اند-که-می‌توانند-هدایت-تیم-ملی-را-به-من-واگذار
------------
2098 title:  مصاحبه فارس با کارشناس فوتبال آسیا| از میراث بزرگ کی‌روش و قدرت ایران با اسکوچیچ تا انقلاب برانکو در عمان 
url:  https://www.farsnews.ir/news/14001124000522/مصاحبه-فارس-با-کارشناس-فوتبال-آسیا|-از-میراث-بزرگ-کی‌روش-و-قدرت-ایران
------------
1466 title:  نکونام: نفتی ها بهترین بازی خود را انجام دادند/بازیکنان جدیدمان کیفیت بالای خود را نشان دادند 
url:  https://www.farsnews.ir/news/14001204001165/نکونام-نفتی-ها-بهترین-بازی-خود-را-انجام-دادند-بازیکنان-جدیدمان-کیفیت
------------
81 title:  ماجدی: فوتبال کشور به تغییرات نیاز دارد 
url:  https://www.farsnews.ir/news/14001223000539/ماجدی-فوتبال-کشور-به-تغییرات-نیاز-دارد
------------
6690 title:  امیدواری ملی پوش سابق فوتبال ساحلی بابت تغییرات در کادرفنی تیم ملی 
url:  https://

In [138]:
query = 'واترپلو'
query_tokens = preprocess([query], True, True)[0]
scores = cosine_similarity(query_tokens, positional_index, len(contents), docs_norms)
print_result(scores, k=5)


1388 title:  تجلیل از خانواده شهید حسن نوفلاح با تقدیم مدال قهرمانی 
url:  https://www.farsnews.ir/news/14001205000938/تجلیل-از-خانواده-شهید-حسن-نوفلاح-با-تقدیم-مدال-قهرمانی
------------
5690 title:  سرپرست  فدراسیون شنا، شیرجه و واترپلو  منصوب شد 
url:  https://www.farsnews.ir/news/14001011000202/سرپرست-فدراسیون-شنا-شیرجه-و-واترپلو-منصوب-شد
------------
5022 title:  رضوانی رئیس فدراسیون شنا ماند/ 3 رای سفید برای تنها کاندیدا 
url:  https://www.farsnews.ir/news/14001020000253/رضوانی-رئیس-فدراسیون-شنا-ماند-3-رای-سفید-برای-تنها-کاندیدا
------------
5013 title:  برنامه‌های رئیس فدراسیون شنا برای 4 سال آینده/ رضوانی: تلاش می‌کنیم به اهداف‌مان در بازی‌های آسیایی برسیم 
url:  https://www.farsnews.ir/news/14001020000394/برنامه‌های-رئیس-فدراسیون-شنا-برای-4-سال-آینده-رضوانی-تلاش-می‌کنیم-به
------------
1300 title:  گرفتن ۷۰ نمونه تست دوپینگ در ۸ رشته طی یک هفته 
url:  https://www.farsnews.ir/news/14001207000351/گرفتن-۷۰-نمونه-تست-دوپینگ-در-۸-رشته-طی-یک-هفته
------------


In [135]:
query = 'واکسن کرونا ایرانی'
query_tokens = preprocess([query], True, True)[0]
scores = cosine_similarity(query_tokens, positional_index, len(contents), docs_norms)
print_result(scores, k=5)

7937 title:  مرندی: رهبر انقلاب دُز سوم واکسن کرونا را دریافت کرده‌اند 
url:  https://www.farsnews.ir/news/14001117000930/مرندی-رهبر-انقلاب-دُز-سوم-واکسن-کرونا-را-دریافت-کرده‌اند
------------
11966 title:  الهیان: وزارت بهداشت درباره علت عدم پیش خرید واکسن فخرا توضیح دهد 
url:  https://www.farsnews.ir/news/14000729000462/الهیان-وزارت-بهداشت-درباره-علت-عدم-پیش-خرید-واکسن-فخرا-توضیح-دهد
------------
7228 title:  رئیسی: مسافرت با رعایت اصول بهداشتی بلااشکال است 
url:  https://www.farsnews.ir/news/14001214000475/رئیسی-مسافرت-با-رعایت-اصول-بهداشتی-بلااشکال-است
------------
9835 title:  نایب رئیس مجلس: دولت در تزریق واکسن و کاهش نگرانی‌‌های مردم شاهکار کرده است 
url:  https://www.farsnews.ir/news/14000923000300/نایب-رئیس-مجلس-دولت-در-تزریق-واکسن-و-کاهش-نگرانی‌‌های-مردم-شاهکار
------------
9736 title:  تأکید مخبر بر حمایت از تولیدکنندگان داخلی واکسن کرونا 
url:  https://www.farsnews.ir/news/14000924000889/تأکید-مخبر-بر-حمایت-از-تولیدکنندگان-داخلی-واکسن-کرونا
------------
